In [24]:
# third-party
import pandas as pd
import scipy
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot, get_relative_errors
from get_data import get_data_by_id_activity
from constants import CATEGORICAL_PALETTE


In [25]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

In [26]:
acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)


Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [27]:
data["1BST"].keys()

dict_keys(['STNB', 'SNB', 'SGB', 'MIXB', 'TR', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'SNBm', 'UALm', 'UARm'])

In [63]:
def plot_resp_param(data, overview, id, activity, sampling_freq=100):

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # plot MAG and airflow signals
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity]['mag']),len(data[id][activity]['mag']))/sampling_freq, 
                             y=data[id][activity]['mag'], 
                             name="MAG"))
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity]['airflow']),len(data[id][activity]['airflow']))/sampling_freq,
                             y=data[id][activity]['airflow'], 
                            name="Airflow", line={"color":"white"}), secondary_y=True)

    # add FR events
    peak_valley = {"TP_i": "valleys", "TP_e": "peaks"}
    print(f"peaks: {overview[id][activity]['MAG']['TP_e']}")
    print(f"valleys: {overview[id][activity]['MAG']['TP_i']}")
    for event in ["TP_i", "TP_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity]["MAG"][event])/sampling_freq, y=data[id][activity]['mag'][overview[id][activity]["MAG"][event]],
                            mode="markers",
                            name=f"TP {peak_valley[event]}"))
        
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity]["Airflow"][peak_valley[event]])/sampling_freq, y=data[id][activity]['airflow'][overview[id][activity]["Airflow"][peak_valley[event]]],
                        mode="markers",
                        name=f"ref {peak_valley[event]}", marker={"color":"white"}), secondary_y=True)
        

    TP_peaks_tuple = [(event, True) for event in overview[id][activity]["MAG"]["TP_e"]]
    TP_valleys_tuple = [(event, True) for event in overview[id][activity]["MAG"]["TP_i"]]
    FN_peaks_tuple = [(event, False) for event in overview[id][activity]["MAG"]["FN_e"]]
    FN_valleys_tuple = [(event, False) for event in overview[id][activity]["MAG"]["FN_i"]]
    extrema = sorted(TP_peaks_tuple + TP_valleys_tuple + FN_peaks_tuple + FN_valleys_tuple)

    first_is_peak = extrema[0][0] in overview[id][activity]["MAG"]["TP_e"] or extrema[0][0] in overview[id][activity]["MAG"]["FN_e"]

    if first_is_peak:
        extrema_pairs_peaks = np.array([[tup1[0], tup2[0], all((tup1[1], tup2[1]))] for tup1, tup2 in zip(
            extrema[::2], extrema[1::2])])
        extrema_pairs_valleys = np.array([[tup1[0], tup2[0], all((tup1[1], tup2[1]))] for tup1, tup2 in zip(
            extrema[1::2], extrema[2::2])])
    else:
        extrema_pairs_peaks = np.array([[tup1[0], tup2[0], all((tup1[1], tup2[1]))] for tup1, tup2 in zip(
            extrema[1::2], extrema[2::2])])
        extrema_pairs_valleys = np.array([[tup1[0], tup2[0], all((tup1[1], tup2[1]))] for tup1, tup2 in zip(
            extrema[::2], extrema[1::2])])
        

    extrema_pairs_peaks = extrema_pairs_peaks[extrema_pairs_peaks[:,2] != 0]
    extrema_pairs_valleys = extrema_pairs_valleys[extrema_pairs_valleys[:,2] != 0]

    for i,ind in enumerate(extrema_pairs_peaks):
        try:
            exp_duration = overview[id][activity]["MAG"]["tE (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+exp_duration, fillcolor=CATEGORICAL_PALETTE[0], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="exp", annotation_position="top left",)
        except Exception as e:
            print(e)

    for i, ind in enumerate(extrema_pairs_valleys):  
        try:
            insp_duration = overview[id][activity]["MAG"]["tI (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+insp_duration, fillcolor=CATEGORICAL_PALETTE[1], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="insp", annotation_position="bottom left",)
        except Exception as e:
            print(e)
                
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

In [66]:
activity="UAR"
id="1BST" 
# not intercalated!!
plot_resp_param(data, overview, id=id, activity=activity)
overview[id][activity]["MAG"]["tI (s)"] - overview[id][activity]["MAG"]["tI airflow (s)"]


peaks: [370, 884, 1309, 1757, 2333, 2864, 3546, 3976, 4393, 4918, 5283, 5543, 6013]
valleys: [133, 536, 1026, 1464, 1957, 2567, 3079, 3812, 4261, 4781, 5098, 5417, 5686]


array([ 0.67,  1.18,  1.27,  1.2 ,  1.09,  1.12,  1.75, -0.18, -0.29,
       -0.7 ,  0.09,  0.06,  1.4 ])

In [55]:
plot_resp_param(data, overview, id="1BST", activity="STNB")

peaks: [242, 654, 1092, 1480, 1916, 2350, 2763, 3202, 3624, 4043, 4455, 4874, 5314, 5722]
valleys: [75, 494, 920, 1294, 1737, 2169, 2590, 3038, 3441, 3865, 4277, 4600, 5126, 5531, 5974]


In [58]:
activity="UAL"
id="1BST"
plot_resp_param(data, overview, id=id, activity=activity)
overview[id][activity]["MAG"]["tI (s)"] - overview[id][activity]["MAG"]["tI airflow (s)"]

peaks: [42, 527, 1012, 1506, 1986, 2469, 2968, 3455, 3921, 4401, 4648, 5161, 5632, 5864]
valleys: [401, 838, 1331, 1827, 2307, 2898, 3311, 3759, 4239, 4489, 5043, 5477, 5926]


array([-1.02,  0.07, -0.55, -0.93, -0.14, -0.88, -0.42, -0.04, -0.18,
        0.08, -0.51, -0.09,  1.97])

In [ ]:
for id in overview.keys():
    for activity in overview[id].keys():
        overview[id][activity]["MAG"]["tI (s)"] - 

### Activity-specific only RP analysis

In [30]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [31]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,0.30 $\pm$ 0.46,14.09 $\pm$ 19.58,0.30 $\pm$ 0.45,11.36 $\pm$ 14.43,0.39 $\pm$ 0.63,8.19 $\pm$ 12.13
1,SNB,PZT,0.57 $\pm$ 0.60,25.61 $\pm$ 31.27,0.58 $\pm$ 0.61,21.89 $\pm$ 17.41,0.44 $\pm$ 0.57,9.53 $\pm$ 11.11
2,SGB,MAG,1.08 $\pm$ 0.88,25.85 $\pm$ 22.45,1.15 $\pm$ 0.84,21.17 $\pm$ 18.40,0.75 $\pm$ 0.73,7.39 $\pm$ 7.35
3,SGB,PZT,1.57 $\pm$ 1.45,25.08 $\pm$ 44.90,1.74 $\pm$ 1.50,29.42 $\pm$ 23.57,0.79 $\pm$ 0.99,7.76 $\pm$ 9.85
4,MIXB,MAG,0.35 $\pm$ 0.44,14.33 $\pm$ 16.50,0.35 $\pm$ 0.46,11.69 $\pm$ 13.70,0.34 $\pm$ 0.43,6.18 $\pm$ 7.21
5,MIXB,PZT,0.78 $\pm$ 0.72,28.08 $\pm$ 26.84,0.86 $\pm$ 1.00,21.19 $\pm$ 32.91,0.70 $\pm$ 0.86,12.74 $\pm$ 15.27
6,STNB,MAG,0.30 $\pm$ 0.52,11.02 $\pm$ 21.02,0.32 $\pm$ 0.49,11.41 $\pm$ 15.89,0.34 $\pm$ 0.58,6.74 $\pm$ 11.18
7,STNB,PZT,0.63 $\pm$ 0.61,23.08 $\pm$ 35.95,0.63 $\pm$ 0.61,21.67 $\pm$ 21.97,0.43 $\pm$ 0.59,8.78 $\pm$ 11.36
8,MCH,MAG,0.40 $\pm$ 0.38,19.84 $\pm$ 19.75,0.35 $\pm$ 0.33,18.82 $\pm$ 17.03,0.34 $\pm$ 0.37,8.95 $\pm$ 9.19
9,MCH,PZT,0.65 $\pm$ 0.57,31.41 $\pm$ 27.86,0.59 $\pm$ 0.50,29.12 $\pm$ 26.75,0.60 $\pm$ 0.57,15.81 $\pm$ 14.68


### Overall RP analysis

In [32]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [33]:
get_breath_parameters_performance(overview_all, target=None)

N=3367 tI for MAG
N=3124 tI for PZT
N=3357 tE for MAG
N=3104 tE for PZT
N=3226 tB for MAG
N=2935 tB for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%),Slope,Intercept,R^2
0,tI,MAG,0.39 $\pm$ 0.44,18.85 $\pm$ 20.87,0.98,0.11,0.64
1,tI,PZT,0.64 $\pm$ 0.63,29.59 $\pm$ 29.32,0.78,0.43,0.44
2,tE,MAG,0.39 $\pm$ 0.43,18.06 $\pm$ 18.06,0.86,0.24,0.71
3,tE,PZT,0.65 $\pm$ 0.66,28.13 $\pm$ 28.10,0.87,0.24,0.59
4,tB,MAG,0.35 $\pm$ 0.46,8.41 $\pm$ 10.33,0.99,0.03,0.88
5,tB,PZT,0.54 $\pm$ 0.63,13.23 $\pm$ 14.91,0.98,0.08,0.82


In [34]:
import plotly.graph_objects as go

for metric in ["tI", "tE", "tB"]:
    rel_error_mag = get_relative_errors(overview_all["MAG"][f"{metric} airflow (s)"], overview_all["MAG"][f"{metric} (s)"])
    rel_error_pzt = get_relative_errors(overview_all["PZT"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"])

    print(f"{metric}: {scipy.stats.mannwhitneyu(rel_error_mag, rel_error_pzt)}")

    fig = go.Figure()
    fig.add_trace(go.Violin(y=rel_error_mag, name="MAG"))
    fig.add_trace(go.Violin(y=rel_error_pzt, name="PZT"))

    # fig.show()

    # normality_test(rel_error_mag, sensor="MAG", type=f"relative error {metric}")
    # normality_test(rel_error_pzt, sensor="PZT", type=f"relative error {metric}")


tI: MannwhitneyuResult(statistic=4961656.5, pvalue=7.978140185968591e-05)
tE: MannwhitneyuResult(statistic=5015347.0, pvalue=0.009338621049301422)
tB: MannwhitneyuResult(statistic=4682829.0, pvalue=0.46167939561389726)


### Bias and variability analysis by activity

In [35]:
from performance import get_bias_variability

get_bias_variability(overview_all_participants, target="Activity", metric="tI", relative_error=False)

,Activity,Sensor,Bias (s),Variability (s)
0,SNB,MAG,0.126,0.535
1,SNB,PZT,0.286,0.778
2,SGB,MAG,0.366,1.351
3,SGB,PZT,-0.369,2.114
4,MIXB,MAG,0.181,0.529
5,MIXB,PZT,0.187,1.045
6,STNB,MAG,0.074,0.601
7,STNB,PZT,0.385,0.791
8,MCH,MAG,0.025,0.552
9,MCH,PZT,-0.013,0.867


In [36]:
activity = "UAL"
for metric in ["tI", "tE", "tB"]:
        bland_altman_plot(overview_all["MAG"][f"{metric} (s)"], overview_all["MAG"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"], overview_all["PZT"][f"{metric} airflow (s)"], metric)
        #bland_altman_plot(overview_all_participants[activity]["MAG"][f"{metric} (s)"], overview_all_participants[activity]["MAG"][f"{metric} airflow (s)"], overview_all_participants[activity]["PZT"][f"{metric} (s)"], overview_all_participants[activity]["PZT"][f"{metric} airflow (s)"], metric, activity)


MAG | mean: 0.067, lloa: -1.07, uloa: 1.20 
PZT | mean: 0.005, lloa: -1.75, uloa: 1.76 


MAG | mean: -0.069, lloa: -1.20, uloa: 1.06 
PZT | mean: -0.048, lloa: -1.86, uloa: 1.76 


MAG | mean: -0.003, lloa: -1.14, uloa: 1.14 
PZT | mean: -0.020, lloa: -1.64, uloa: 1.60 
